In [1]:
print('Hola Mundo')

Hola Mundo


In [2]:
import pandas as pd
import numpy as np

from ydata_profiling import ProfileReport
import glob
import seaborn as sns

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
csv_files = glob.glob('MLOpsReviews/ratings/*.csv')
df_list = []
for csv_file in csv_files:
    df_ratings = pd.read_csv(csv_file)
    df_list.append(df_ratings)
df_ratings = pd.concat(df_list)

In [4]:
df = pd.read_csv('df.csv')

In [5]:
df_ratings.shape

(11024289, 5)

In [ ]:
ProfileReport(df)

In [ ]:
ProfileReport(df_ratings)

In [5]:
df_ratings = df_ratings.drop_duplicates()

In [19]:
df_ratings.describe()

,userId,rating,timestamp
count,5.999935e+06,5.999935e+06,5.999935e+06
mean,3.102307e+04,3.527982e+00,1.173530e+09
std,1.785850e+04,1.058264e+00,2.055337e+08
min,1.000000e+00,5.000000e-01,7.896520e+08
25%,1.542400e+04,3.000000e+00,9.921795e+08
50%,3.133100e+04,3.500000e+00,1.154880e+09
75%,4.620000e+04,4.000000e+00,1.362442e+09
max,6.176800e+04,5.000000e+00,1.501827e+09


In [6]:
#cambio el nombre de la columna
df_ratings = df_ratings.rename(columns={'movieId': 'id'})

In [7]:
#uno los dos dataset, y solo me quedo con las columnas que me interesan para mi modelo
rated_movie= df_ratings.merge(df,left_on='id', right_on='id')[['userId','rating_x','id','title']]

In [ ]:
rated_movie.sample(10)

In [8]:
##aca veo, la película, la cantidad de veces que la puntuaron y el promedio de la ppuntuacion
mean_ratings=rated_movie.pivot_table(index='title', values='rating_x', aggfunc=[len, np.mean])

In [ ]:
mean_ratings.head()

In [9]:
mean_ratings.columns =['ratings','mean_ratings']

In [ ]:
mean_ratings.head()

In [13]:
mean_ratings = pd.DataFrame(mean_ratings)
mean_ratings.to_csv("promedios.csv")

MODELO PREDICTIVO

In [24]:
#vamos a ver cuales son las pelis con mejor puntaje
mean_ratings.sort_values(by='mean_ratings', ascending=False)

,ratings,mean_ratings
title,,
thalaivii (telugu),242,3.822314
master (tamil),265,3.771698
brightheart: let your light shine,273,3.767399
spirit rider,273,3.761905
tuhya dharma koncha,283,3.761484
...,...,...
roll with me,227,3.288546
ex machina,216,3.287037
"thomas & friends: the great race (us english, brazilian portuguese, latin america spanish)",253,3.264822


In [12]:
#vamos a ver lo mismo, pero filtrando por cantidad de veces que fue puntuada
mean_ratings[mean_ratings['ratings'] > 300].sort_values(by='mean_ratings', ascending=False).head(5)

#puse la condicion de que ratings sea mayor a 300 para asegurarme que sea una peli muy vista, más popular

,ratings,mean_ratings
title,,
thalaivii (telugu),461,3.724512
the man who killed hitler and then the bigfoot,525,3.717143
space jungle,485,3.709278
garden of hedon,430,3.708140
the haunting in connecticut,490,3.703061


In [ ]:
##FILTRADO COLABORATIVO

rated_movie.shape

In [ ]:
#voy a trabajar con un dataset que tenga como filas los usuarios y como colummnas las peliculas
ratings_pivot=rated_movie.pivot_table(index='userId', columns='title',values='rating_x', aggfunc=np.mean) #los últimos parámetros son en caso de que 
#un usuario haya votado la misma pelicula más d euna vez

#ACA YA TENGO PROBLEMAS, ME TIRA ERROR PORQUE NO TENGO LA MEMORIA RAM SUFICIENTE PARA TIRAR ESTO

#PODRIA TOMAR PARA LA MATRIZ DE DISPERSION SOLO 5 MIL REGISTROS, LO CUAL ME PARECE INEFICIENTE, PORQUE NO PUEDO ENTRENAR UN MODELO DE 
#11 MILLONES DE REGISTROS, CON SOLO 5 MIL. POR LO TANTO, DEJO EL CODIGO DE CÓMO LO SEGURÍA TRABAJANDO ASÍ DE USUARIO A ITEM, 
#PERO NO LO VOY A EJECUTAR MÁS PORQUE ME VA A EXPLOTAR LA COMPU

In [ ]:
#EN UN MUNDO IDEAL, DONDE MI MÁQUINA PODRÍA EJECUTAR EL CODIGO ANTERIOR, ME HUBIESE QUEDADO UNA MATRIZ DE 11024289X22042
#CON MUCHOS VALORES NULOS, POR LO QUE COMPLETARIA ESOS VALORES NULOS CON CEROS:

ratings_pivot.fillna(0, inplace=True)

In [ ]:
#LO CONVIERTO A UNA MATRIZ DE DISPERSION:
sparse_rating= sp.sparce.csr_matrix(ratings_pivot.values)

In [ ]:
#SOBRE ESTA MATRIZ DISPERSA, AHORA VOY A COMPARAR SIMILITUDES: DISTANCIA SENO Y DISTANCIA COSENO, para agregar puntajes a los valores faltantes
user_similarity= cosine_similarity(sparse_rating)  #SIMILITUD ENTRE LOS USUARIOS
item_similarity= cosine_similarity(sparse_rating.T) #SIMILITUD ENTRE LAS PELICULAS, USANDO LA MATRIZ TRASPUESTA


In [ ]:
user_similarity_df = pd.DataFrame(user_similarity,index=ratings_pivot.index, columns=ratings_pivot.index)
item_similarity_df= pd.DataFrame(item_similarity,index=ratings_pivot.columns, columns=ratings_pivot.columns)

In [ ]:
item_similarity_df['mi pelicula'].sort_values(ascending=False)